In [ ]:
# Step 1. Mount drive 
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Step 2. Tai ma nguon YOLO ve drive
!rm -rf darknet
%cd /content/gdrive/MyDrive
!git clone https://github.com/AlexeyAB/darknet
%cd /content/gdrive/MyDrive/darknet
!rm -rf data
!mkdir data

In [ ]:
#check cuda version
!nvidia-smi -L
!nvcc -V

In [ ]:
# Step 4. Giải nén file data
%cd /content/gdrive/MyDrive/YoloV3/
!unzip /content/gdrive/MyDrive/YoloV3/darknet.zip

In [ ]:
%cd /content/gdrive/MyDrive/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# Step 5. Tạo file yolo.names
%cd /content/gdrive/MyDrive/darknet
!echo "table" > yolo.names
!echo "image" >> yolo.names
!echo "text" >> yolo.names
!echo "math" >> yolo.names

In [ ]:
# Step 6. Tạo file train.txt và val.txt
%cd /content/gdrive/MyDrive/darknet

import glob2
import math  
import os
import numpy as np

files = []
for ext in ["*.png", "*.jpeg", "*.jpg"]:
  image_files = glob2.glob(os.path.join("data/data/", ext))
  files += image_files

nb_val = math.floor(len(files)*0.2)
rand_idx = np.random.randint(0, len(files), nb_val)

# Tạo file train.txt
with open("train.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')

# Tạo file vali.txt
with open("val.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (idx in rand_idx) and (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')

In [ ]:
# Step 7. Tạo file yolo.data
%cd /content/gdrive/MyDrive/darknet
!mkdir backup
!echo classes=4 > yolo.data
!echo train=train.txt >> yolo.data
!echo valid=val.txt >> yolo.data
!echo names=yolo.names >> yolo.data
!echo backup=backup >> yolo.data

In [ ]:
# Step 8. Make darknet
%cd /content/gdrive/MyDrive/darknet
!find . -type f | xargs -n 5 touch
!rm darknet
!make clean
!make

In [ ]:
# Step 9. Download pretrain weight
%cd /content/gdrive/MyDrive/darknet
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
# Step 10. Train
################--Train--################
%cd /content/gdrive/MyDrive/darknet
!./darknet detector train yolo.data cfg/yolov4.cfg yolov4.conv.137 -dont_show

In [ ]:
# Step 10. Train continue
%cd /content/gdrive/MyDrive/darknet
!./darknet detector train yolo.data cfg/yolov4.cfg backup/yolov4_last.weights -dont_show

In [ ]:
import cv2
import numpy as np
import time
from matplotlib import pyplot as plt

img = cv2.imread("/content/gdrive/MyDrive/YoloV3/darknet/data/images/image-001.jpg")

width = img.shape[1]
height = img.shape[0]
yolo_w = 416
yolo_h = 416
VEHICLE_CLASSES = [1, 2]

classes = ["math", "image", "table", "text"]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

net = cv2.dnn.readNetFromDarknet("/content/gdrive/MyDrive/darknet/cfg/yolov4.cfg", "/content/gdrive/MyDrive/darknet/backup/yolov4_last.weights")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
# construct a blob from the image
blob = cv2.dnn.blobFromImage(img, 1/255.0, (yolo_w, yolo_h), swapRB=True, crop=False)

net.setInput(blob)

outputs = net.forward(ln)

for out in outputs:
    print(out.shape)

boxes = []
class_ids = []
confidences = []

for out in outputs:
  for detection in out:
    scores = detection[5:]
    class_id = np.argmax(scores)
    confidence = scores[class_id]
    if confidence > 0.6:
        center_x = int(detection[0] * width)
        center_y = int(detection[1] * height)
        w = int(detection[2] * width)
        h = int(detection[3] * height)
        x = int(center_x - w / 2)
        y = int(center_y - h / 2)
        class_ids.append(class_id)
        confidences.append(float(confidence))
        boxes.append([x, y, w, h])